In [1]:
import numpy as np
from glob import glob
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions

In [2]:
source_dir = "G:/BracU/Thesis/Federated Dataset/PlantVillage/Train"
test_dir = "G:/BracU/Thesis/Federated Dataset/PlantVillage/Test"
client_dirs = glob(source_dir+ "/*")

In [3]:
input_shape = (128, 128)
batch_size = 32

generators_train_client = []
generators_val_client = []

for i in range(0, len(client_dirs)):
    datagen_train = ImageDataGenerator(rescale=1./255, validation_split=0.1)
    generator_train = datagen_train.flow_from_directory(directory=client_dirs[i],
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    class_mode='sparse',
                                                    subset="training",
                                                    shuffle=True)
    generator_val = datagen_train.flow_from_directory(directory=client_dirs[i],
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    subset="validation",
                                                    class_mode='sparse',
                                                    shuffle=True)
    generators_train_client.append(generator_train)
    generators_val_client.append(generator_val)
num_classes = generators_train_client[0].num_classes

Found 4946 images belonging to 15 classes.
Found 544 images belonging to 15 classes.
Found 4948 images belonging to 15 classes.
Found 544 images belonging to 15 classes.
Found 4935 images belonging to 15 classes.
Found 541 images belonging to 15 classes.


In [4]:
datagen_test = ImageDataGenerator(rescale=1./255)
generators_test = datagen_train.flow_from_directory(directory=test_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    class_mode='sparse',
                                                    shuffle=True)

Found 4180 images belonging to 15 classes.


In [5]:
from tensorflow.keras.layers import Layer, BatchNormalization, ReLU, Input


class Involution(Layer):
    def __init__(self, channels, kernel_size=3, reduction_ratio=2, group_channels=1, **kwargs):
        super(Involution, self).__init__(**kwargs)  # Pass kwargs to the base Layer class
        self.channels = channels
        self.kernel_size = kernel_size
        self.reduction_ratio = reduction_ratio
        self.group_channels = group_channels
        self.groups = channels // group_channels

    def build(self, input_shape):
        self.reduce_channels = self.channels // self.reduction_ratio
        self.kernel_gen = tf.keras.Sequential([
            tf.keras.layers.Conv2D(self.reduce_channels, kernel_size=1, padding='same', use_bias=False),
            BatchNormalization(),
            ReLU(),
            tf.keras.layers.Conv2D(self.kernel_size * self.kernel_size * self.groups, kernel_size=1, padding='same')
        ])

    def call(self, inputs):
        batch_size, height, width, in_channels = tf.shape(inputs)[0], tf.shape(inputs)[1], tf.shape(inputs)[2], tf.shape(inputs)[3]
        
        # Generate kernels
        kernels = self.kernel_gen(inputs)
        kernels = tf.reshape(kernels, (batch_size, height, width, self.kernel_size * self.kernel_size, self.groups))
        kernels = tf.nn.softmax(kernels, axis=-2)

        # Unfold input
        x_unfold = tf.image.extract_patches(inputs, sizes=[1, self.kernel_size, self.kernel_size, 1], 
                                            strides=[1, 1, 1, 1], rates=[1, 1, 1, 1], padding='SAME')
        x_unfold = tf.reshape(x_unfold, (batch_size, height, width, self.kernel_size * self.kernel_size, self.groups))

        # Apply involution
        outputs = tf.reduce_sum(kernels * x_unfold, axis=-2)
        outputs = tf.reshape(outputs, (batch_size, height, width, self.channels))
        
        return outputs

    def get_config(self):
        config = super().get_config()
        config.update({
            'channels': self.channels,
            'kernel_size': self.kernel_size,
            'reduction_ratio': self.reduction_ratio,
            'group_channels': self.group_channels,
        })
        return config

In [6]:
def build_invnet_vgg16_optimized(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    
    # Block 1
    x = Involution(channels=3, kernel_size=3)(inputs)
    x = Involution(channels=3, kernel_size=3)(x)
    x = ReLU()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    
    # Block 2
    x = Involution(channels=3, kernel_size=3)(x)
    x = Involution(channels=3, kernel_size=3)(x)
    x = ReLU()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    
    # Block 3
    x = Involution(channels=3, kernel_size=3)(x)
    x = Involution(channels=3, kernel_size=3)(x)
    x = ReLU()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    
    # Block 4
    x = Involution(channels=3, kernel_size=3)(x)
    x = ReLU()(x)
    x = Involution(channels=3, kernel_size=3)(x)
    x = ReLU()(x)
    x = Involution(channels=3, kernel_size=3)(x)
    x = ReLU()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    
    # Fully connected layers
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    return model

# Define input shape and number of classes
input_shape = (128, 128, 3)
num_classes = 17

opt = Adam(0.001)

# Build and compile the InvNet model
model = build_invnet_vgg16_optimized(input_shape, num_classes)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 involution (Involution)     (None, 128, 128, 3)       61        
                                                                 
 involution_1 (Involution)   (None, 128, 128, 3)       61        
                                                                 
 re_lu (ReLU)                (None, 128, 128, 3)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 3)        0         
 )                                                               
                                                                 
 involution_2 (Involution)   (None, 64, 64, 3)         61        
                                                             

In [8]:
# Build the initial global model
global_model = build_invnet_vgg16_optimized(input_shape, num_classes)
global_model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Simulate federated learning process



num_clients = len(client_dirs)
num_rounds = 50
local_epochs = 1

# List to store local models
local_models = []

for round_num in range(num_rounds):
    print(f"Round {round_num + 1}/{num_rounds}...")
    local_models = []  # Reset local models for each round
    for client in range(num_clients):
        print(f"Training local model {client + 1}/{num_clients}...")
        local_model = tf.keras.models.clone_model(global_model)  # Clone the global model structure
        local_model.set_weights(global_model.get_weights())  # Initialize with global model weights
        
        # Compile the local model
        local_model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        # Train the local model
        local_model.fit(generators_train_client[client], epochs=local_epochs, validation_data=generators_val_client[client])
        local_models.append(local_model)

    # Aggregating weights (Federated Averaging)
    new_weights = [np.zeros_like(weight) for weight in global_model.get_weights()]

    for local_model in local_models:
        local_weights = local_model.get_weights()
        for i in range(len(new_weights)):
            new_weights[i] += local_weights[i] / num_clients

    global_model.set_weights(new_weights)

# Evaluate the global model
print("Evaluating the global model...")
global_model.evaluate(generators_test)


Round 1/50...
Training local model 1/3...
155/155 [==============================] - 17s 92ms/step - loss: 2.6311 - accuracy: 0.1442 - val_loss: 2.5648 - val_accuracy: 0.1562
Training local model 2/3...
155/155 [==============================] - 18s 97ms/step - loss: 2.6402 - accuracy: 0.1431 - val_loss: 2.5590 - val_accuracy: 0.1691
Training local model 3/3...
155/155 [==============================] - 17s 90ms/step - loss: 2.6316 - accuracy: 0.1443 - val_loss: 2.5456 - val_accuracy: 0.1590
Round 2/50...
Training local model 1/3...
155/155 [==============================] - 18s 97ms/step - loss: 2.5752 - accuracy: 0.1535 - val_loss: 2.5571 - val_accuracy: 0.1636
Training local model 2/3...
155/155 [==============================] - 17s 90ms/step - loss: 2.5805 - accuracy: 0.1556 - val_loss: 2.5660 - val_accuracy: 0.1636
Training local model 3/3...
155/155 [==============================] - 17s 90ms/step - loss: 2.5657 - accuracy: 0.1576 - val_loss: 2.5469 - val_accuracy: 0.1664
Round 

KeyboardInterrupt: 